In [2]:
from llama_index import (
    SimpleDirectoryReader,
    ServiceContext,
    KnowledgeGraphIndex,
    SummaryIndex,
)
from llama_index.graph_stores import SimpleGraphStore
import torch

from llama_index.llms import OpenAI, HuggingFaceLLM
from llama_index.prompts import PromptTemplate
from transformers import BitsAndBytesConfig

from IPython.display import Markdown, display

/home/alex/.local/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.2) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [3]:
documents = SimpleDirectoryReader(
    "../ingestion/local_files/"
).load_data()

In [5]:
def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}</s>\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}</s>\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}</s>\n"

  # ensure we start with a system prompt, insert blank if needed
  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n</s>\n" + prompt

  # add final assistant prompt
  prompt = prompt + "<|assistant|>\n"

  return prompt

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
llm = HuggingFaceLLM(
    model_name="HuggingFaceH4/zephyr-7b-alpha",
    tokenizer_name="HuggingFaceH4/zephyr-7b-alpha",
    query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.1, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)

service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:26                                                                                   │
│                                                                                                  │
│   23 │   bnb_4bit_quant_type="nf4",                                                              │
│   24 │   bnb_4bit_use_double_quant=True,                                                         │
│   25 )                                                                                           │
│ ❱ 26 llm = HuggingFaceLLM(                                                                       │
│   27 │   model_name="HuggingFaceH4/zephyr-7b-alpha",                                             │
│   28 │   tokenizer_name="HuggingFaceH4/zephyr-7b-alpha",                                         │
│   29 │   query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|    │
│                                                                                                  │
│ /home/alex/.local/lib/python3.10/site-packages/llama_index/llms/huggingface.py:131 in __init__   │
│                                                                                                  │
│   128 │   │   │   ) from exc                                                                     │
│   129 │   │                                                                                      │
│   130 │   │   model_kwargs = model_kwargs or {}                                                  │
│ ❱ 131 │   │   self._model = model or AutoModelForCausalLM.from_pretrained(                       │
│   132 │   │   │   model_name, device_map=device_map, **model_kwargs                              │
│   133 │   │   )                                                                                  │
│   134                                                                                            │
│                                                                                                  │
│ /home/alex/.local/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:565 in   │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   562 │   │   │   )                                                                              │
│   563 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   564 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 565 │   │   │   return model_class.from_pretrained(                                            │
│   566 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   567 │   │   │   )                                                                              │
│   568 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ /home/alex/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:3307 in            │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   3304 │   │   │   │   mismatched_keys,                                                          │
│   3305 │   │   │   │   offload_index,                                                            │
│   3306 │   │   │   │   error_msgs,                                                               │
│ ❱ 3307 │   │   │   ) = cls._load_pretrained_model(                                               │
│   3308 │   │   │   │   model,                              

In [13]:

summary_index = SummaryIndex.from_documents(documents, service_context=service_context)

0.8.48
